In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#input_file = "output_20191008_0336AM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #double
df = pd.read_csv(input_file, header = 0)

In [3]:
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']

In [4]:
df_failure = df[ (df.FM==1)]

In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.calibration import CalibratedClassifierCV

In [8]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])

In [15]:
y=df_failure['LABEL']
x=df_dummy.drop(['LABEL'],axis=1)

In [17]:
linear_svc=svm.LinearSVC(random_state=0)
lin_clf=linear_svc.fit(x, y)

In [18]:
cal_svc=CalibratedClassifierCV(base_estimator=linear_svc,cv="prefit")
cal_clf=cal_svc.fit(x,y)

In [19]:
%%time
ovo_lin_model=OneVsOneClassifier(svm.LinearSVC(random_state=0)).fit(x, y)

CPU times: user 10.5 s, sys: 43.6 ms, total: 10.6 s
Wall time: 10.6 s


In [20]:
%%time
#svm_pred=svm_model.predict(x)
lin_df = ovo_lin_model.decision_function(x)

CPU times: user 52.5 s, sys: 1.45 s, total: 53.9 s
Wall time: 17.9 s


In [21]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
    return correct, correct/num

In [30]:
c,r=accuracy(cal_clf, df_dummy, ovo_lin_model.classes_,1)
c,r

(40, 0.6060606060606061)

In [32]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score
y_pred_rf = cal_clf.predict(x)
print(balanced_accuracy_score(y, y_pred_rf))
f1_score(y, y_pred_rf,average='weighted')

0.2463278028318829


/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3680773409448971

In [ ]:
c,r=accuracy(lin_df, df_dummy, ovo_lin_model.classes_,10)
c,r

In [ ]:
ovo_svm_model=OneVsOneClassifier(svm.SVC(random_state=0)).fit(x, y)
#svm_pred=svm_model.predict(x)
svm_df = ovo_svm_model.decision_function(x)

In [14]:
def accuracy_ori(model, test_data, classes):
    num = classes.size
    correct=0
    for label in classes:
        test_data_1=test_data[test_data.LABEL==label]
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.decision_function(x_test_1)
        label_index=test_prob_1.sum(axis=0).argmax()
        label_pred = classes[label_index]
        if(label==label_pred):
            correct=correct+1
    return correct, correct/num

In [15]:
c,r=accuracy_ori(lin_df, df_dummy, ovo_lin_model.classes_)
c,r

AttributeError: 'numpy.ndarray' object has no attribute 'decision_function'

In [106]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [107]:
multilabel_confusion_matrix(y, svm_pred)

array([[[26755,     0],
        [  156,     0]],

       [[26252,   118],
        [   95,   446]],

       [[26589,     0],
        [  322,     0]],

       [[26794,     0],
        [  117,     0]],

       [[26061,    31],
        [  386,   433]],

       [[25767,   707],
        [  238,   199]],

       [[25509,   277],
        [  633,   492]],

       [[26900,     0],
        [   11,     0]],

       [[25812,   732],
        [  158,   209]],

       [[26900,     0],
        [   11,     0]],

       [[26900,     0],
        [   11,     0]],

       [[25994,   535],
        [  190,   192]],

       [[24880,  1061],
        [  155,   815]],

       [[26900,     0],
        [   11,     0]],

       [[26013,   271],
        [  512,   115]],

       [[26479,    92],
        [  286,    54]],

       [[26085,   416],
        [  272,   138]],

       [[26900,     0],
        [   11,     0]],

       [[26900,     0],
        [   11,     0]],

       [[25889,   635],
        [  212,   175]],



In [108]:
cross_val_score(clf_svm, x, y, scoring='accuracy',cv=5)

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

array([0.08759933, 0.23890848, 0.40580249, 0.41075749, 0.19336192])

In [109]:
cross_val_score(clf_linsvm, x, y, scoring='accuracy',cv=5)

array([0.09850305, 0.24633377, 0.4041287 , 0.42453006, 0.19541302])

In [110]:
f1_score(y, lin_pred,average='weighted')

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.39628869775904857

In [111]:
f1_score(y, svm_pred,average='weighted')

0.4043405159517015

In [112]:
clf_svm.score(x,y)

0.42499349708297723

In [113]:
clf_linsvm.score(x,y)

0.4150347441566646

In [114]:
balanced_accuracy_score(y, svm_pred)

0.2803607623540877

In [115]:
balanced_accuracy_score(y, lin_pred)

0.2877566726729404

In [129]:
x.values.shape

(26911, 26)

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

In [ ]:
classifier = BinaryRelevance(classifier = SVC(probability=True),
    require_dense = [False, True])

classifier.fit(x.to_numpy(), y.to_numpy())